# Extreme multi-class classification

ID2223 Scalable Machine Learning and Deep Learning

**Federico Baldassarre (fedbal@kth.se) and Beatrice Ionascu (bionascu@kth.se)**

In [ ]:
%matplotlib inline

import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

plt.rcParams['figure.figsize'] = (15, 5)

# add the 'src' directory so we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

from utils.paths import data_raw_dir
from utils.paths import data_processed_dir

## Problem
The goal of this project is to classify products from [Cdiscount](https://www.cdiscount.com/), France’s largest non-food e-commerce company, based on the product images on the company's website. Being able to correctly predict the category of products is important in ensuring that new products are well classified. 

## Data set

![data](../figures/data.png)

The Cdiscount dataset consists of 15 million images at 180x180 resolution of almost 9
million products. The training data consists of a list of 7,069,896 dictionaries, one per product. Each dictionary contains a product id, the category id of the product, and between
1-4 images, stored in a list. In addition, each category id has a corresponding level1,
level2, and level3 name, in French.

In [ ]:
categories = pd.read_pickle(os.path.join(data_processed_dir, 'categories.pickle'))
display(HTML(categories.filter(like='category', axis=1).head().to_html(index=False)))

In [ ]:
product_distrib = pd.read_pickle(os.path.join(data_processed_dir, 'cat_id_prod_distrib.pickle'))
print('There are {} products.'.format(product_distrib.sum()))

In [ ]:
image_distrib = pd.read_pickle(os.path.join(data_processed_dir, 'cat_id_img_distrib.pickle'))
print('There are {} images.'.format(image_distrib.sum()))

In [ ]:
categ_counts = pd.read_csv(os.path.join(data_raw_dir, 'category_names.csv'))
print(categ_counts.nunique().to_frame('Category counts'))

The 5270 available categories are very unevenly distributed amongst the products. As seen below, there are many categories with just a few products and few categories with very many products.

In [ ]:
cat_id_distrib = pd.read_pickle(os.path.join(data_processed_dir, 'cat_id_prod_distrib.pickle'))
cat_1_distrib = pd.read_pickle(os.path.join(data_processed_dir, 'cat_1_prod_distrib.pickle'))
cat_2_distrib = pd.read_pickle(os.path.join(data_processed_dir, 'cat_2_prod_distrib.pickle'))
cat_3_distrib = pd.read_pickle(os.path.join(data_processed_dir, 'cat_3_prod_distrib.pickle'))
fig, ax = plt.subplots(2, 2, figsize=(16,10))
ax=ax.ravel()
cat_1_distrib.hist(log=True, bins=50,ax=ax[0])
ax[0].set_title('Category Level 1 ({})'.format(cat_1_distrib.size)); ax[0].set_xlabel('Number of products')
cat_2_distrib.hist(log=True, bins=50,ax=ax[1])
ax[1].set_title('Category Level 2 ({})'.format(cat_2_distrib.size)); ax[1].set_xlabel('Number of products')
cat_3_distrib.hist(log=True, bins=50,ax=ax[2])
ax[2].set_title('Category Level 3 ({})'.format(cat_3_distrib.size)); ax[2].set_xlabel('Number of products')
cat_id_distrib.hist(log=True, bins=50,ax=ax[3])
ax[3].set_title('Category id ({})'.format(cat_id_distrib.size)); ax[3].set_xlabel('Number of products')

The categories with the most and least products are the following:

In [ ]:
pd.merge(categories.filter(like='category'), 
         cat_id_distrib.sort_values(ascending=False).iloc[np.r_[0:5, -5:0]].to_frame('counts'),
         left_index=True, right_index=True).sort_values('counts', ascending=False)

## Approach
### Transfer Learning
It's been proved, both in the literature and by practical applications, that an image classification task can benefit from transfer learning. They key concept is to reuse the architecture and the weights of another network to extract a representation of the images, which hopefully contains useful information for the classification task. Once an images have been embedded into a compressed feature space, a shallow network can be used to classify them into the final categories.

The whole network can be sketched as:

![xception+shallow network](../figures/architecture.png)

During training the weights of the feature extraction network are frozen, reducing the weights to learn to only those of the shallow network at the end.

The advantages of transfer learning are enabled by the availability of models trained on a very large and general image datasets and released to the public. In fact, training those models can take a long time (up to a few weeks) even in a distributed GPU environment, making it unfeasible to train them jointly with the shallow network at the end. Also, their depth would provide a challenge in terms of gradient propagation, especially at the beginning when the last layers try to classify an image on the base of an embedding that is almost random and therefore are unable to provide the previous layers with useful information on how to update their weights.

We have chosen the recent [Xception model](https://arxiv.org/abs/1610.02357) for feature extraction over the more popular [VGG](https://arxiv.org/abs/1409.1556) or [Inception](https://arxiv.org/abs/1409.4842) models. The choice is motivated by the former being lighter and faster than the other two, achieving almost the same accuracy on the ImageNet benchmark ([comparison](https://keras.io/applications#documentation-for-individual-models)).

### Classification network architecture and negative sampling
The architecture we used for the classification network is composed of three convolutional layers with droupout and a fully connected softmax layer. The purpose of the first 1x1 convolution is to reduce the depth of the embedding volume, before moving further down the network. The output of the final fully connected layer represents the probability distribution over the classes.

```
_________________________________________________________________
Layer (type)                 Output Shape                Param #   
=================================================================
InputLayer                   (?, 6, 6, 2048)                 
_________________________________________________________________
Conv2D (1x1 kernel)          (?, 6, 6, 64)               131,136    
_________________________________________________________________
Dropout                      (?, 6, 6, 64)                   
_________________________________________________________________
Conv2D (4x4 kernel)          (?, 3, 3, 128)              131,200    
_________________________________________________________________
Dropout                      (?, 3, 3, 128)                  
_________________________________________________________________
Conv2D (3x3 kernel)          (?, 1, 1, 256)              295,168    
_________________________________________________________________
Dropout                      (?, 1, 1, 256)                  
________________________________________________________________
Flatten                      (?, 256)                        
_________________________________________________________________
Dense                        (?, 5270)                 1,354,390   
=================================================================
Trainable params: 1,911,894
_________________________________________________________________
```

It is evident that in such architecture the main contribution to the number of trainable parameters comes from the fully connected layer. In fact, it is accountable for approx. 70% of the weights of the network. The reason for this is the high number of probability values to output (5270 versus the 1000 classes of ImageNet). 

Training a network with a large number of parameters turns out to be slow, but we can employ a trick from Natural Language Processing to improve the training speed. Instead of computing the cross entropy loss for all the 5270 output probabilities we can subsample the output vector to include the true label and part of the false labels and only compute the loss for these. In this way we backpropagate fewer gradients through the last layer and reduce the computational effort, effectively speding up training. Of course, the price to pay for subsampling the loss function is less informative gradients and lower final accuracy.

TODO maybe image

## Pre-processing
### Label encoding

## Feature Extraction
### TfRecords

### Computation graph
![computation graph](../figures/computational_graph.png)

## Training

- We trained the network using two different loss functions.
- explain training and testing split
- explain epoch / step/batch size


## Results


![results](../figures/metrics_v2.png)

- classification accuracy and loss per image
- can't test on the test data since we don't have access to it ...



#### Future work

- group images into products -- report results per product
- test the influence of the negative sample size in sampled softmax
- use heirarchy
- use OCR for books
- ...